## ewf-wfp-02-01-01 - Smoothed Gap-Filled Land Surface Temperature Time Series

Smoothed Gap-Filled Land Surface Temperature Time Series

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Smoothed Gap-Filled Land Surface Temperature Time Series'),
                ('abstract', 'Smoothed Gap-Filled Land Surface Temperature Time Series'),
                ('id', 'ewf-wfp-02-01-01')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the MDOIS stack of products' identifiers

In [ ]:
# 2015
input_identifiers = ('MOD11C2.A2015001.006.2016223171153', 'MOD11C2.A2015009.006.2016223171154', 'MOD11C2.A2015017.006.2016223171154', 'MOD11C2.A2015025.006.2016223171154', 'MOD11C2.A2015033.006.2016223171154')

#input_identifiers = ('MOD11C2.A2015001.006.2016223171153','MOD11C2.A2015009.006.2016223171154','MOD11C2.A2015017.006.2016223171154','MOD11C2.A2015025.006.2016223171154','MOD11C2.A2015033.006.2016223171154','MOD11C2.A2015041.006.2016223171218','MOD11C2.A2015049.006.2016223171218','MOD11C2.A2015057.006.2016223171220','MOD11C2.A2015065.006.2016223171152','MOD11C2.A2015073.006.2016223171152','MOD11C2.A2015081.006.2016223171158','MOD11C2.A2015089.006.2016223171156')

# 2016
#input_identifiers = ('MOD11C2.A2016001.006.2016234032340', 'MOD11C2.A2016009.006.2016234032341', 'MOD11C2.A2016017.006.2016234032342', 'MOD11C2.A2016025.006.2016234032341', 'MOD11C2.A2016033.006.2016235185021')

# 2017
input_identifiers = ( 'MOD11C2.A2017001.006.2017013160908', 'MOD11C2.A2017009.006.2017020191355', 'MOD11C2.A2017017.006.2017025093208', 'MOD11C2.A2017025.006.2017033125003', 'MOD11C2.A2017033.006.2017041092627')

## 2018
#input_identifiers = ( 'MOD11C2.A2018001.006.2018011143504', 'MOD11C2.A2018009.006.2018017084533', 'MOD11C2.A2018017.006.2018025152434', 'MOD11C2.A2018025.006.2018033090239', 'MOD11C2.A2018033.006.2018041083108')

#input_identifiers = ('MOD11C2.A2015001.006.2016223171153','MOD11C2.A2015009.006.2016223171154','MOD11C2.A2015017.006.2016223171154','MOD11C2.A2015025.006.2016223171154','MOD11C2.A2015033.006.2016223171154','MOD11C2.A2015041.006.2016223171218','MOD11C2.A2015049.006.2016223171218','MOD11C2.A2015057.006.2016223171220','MOD11C2.A2015065.006.2016223171152','MOD11C2.A2015073.006.2016223171152','MOD11C2.A2015081.006.2016223171158','MOD11C2.A2015089.006.2016223171156','MOD11C2.A2015097.006.2016223171155','MOD11C2.A2015105.006.2016223171201','MOD11C2.A2015113.006.2016223171157','MOD11C2.A2015121.006.2016223171155','MOD11C2.A2015129.006.2016223171158','MOD11C2.A2015137.006.2016223171158','MOD11C2.A2015145.006.2016223171151','MOD11C2.A2015153.006.2016223171150','MOD11C2.A2015161.006.2016223171158','MOD11C2.A2015169.006.2016223171150','MOD11C2.A2015177.006.2016223171150','MOD11C2.A2015185.006.2016223171220','MOD11C2.A2015193.006.2016224093011','MOD11C2.A2015201.006.2016224093008','MOD11C2.A2015209.006.2016224093012','MOD11C2.A2015217.006.2016224093008','MOD11C2.A2015225.006.2016227013030','MOD11C2.A2015233.006.2016227013023','MOD11C2.A2015241.006.2016227013026','MOD11C2.A2015249.006.2016227013025','MOD11C2.A2015257.006.2016228040121','MOD11C2.A2015265.006.2016228040124','MOD11C2.A2015273.006.2016228040124','MOD11C2.A2015281.006.2016228040123','MOD11C2.A2015289.006.2016229085710','MOD11C2.A2015297.006.2016229085710','MOD11C2.A2015305.006.2016229085705','MOD11C2.A2015313.006.2016229085710','MOD11C2.A2015321.006.2016230144838','MOD11C2.A2015329.006.2016230144835','MOD11C2.A2015337.006.2016230144832','MOD11C2.A2015345.006.2016230144837','MOD11C2.A2015353.006.2016231004440','MOD11C2.A2015361.006.2016234031745','MOD11C2.A2016001.006.2016234032340','MOD11C2.A2016009.006.2016234032341','MOD11C2.A2016017.006.2016234032342','MOD11C2.A2016025.006.2016234032341','MOD11C2.A2016033.006.2016235185021','MOD11C2.A2016041.006.2016235185017','MOD11C2.A2016049.006.2016235185116','MOD11C2.A2016057.006.2016235185327','MOD11C2.A2016065.006.2016242200752','MOD11C2.A2016073.006.2016242200756','MOD11C2.A2016081.006.2016242200800','MOD11C2.A2016089.006.2016242201206','MOD11C2.A2016097.006.2016242201205','MOD11C2.A2016105.006.2016242201202','MOD11C2.A2016113.006.2016242201629','MOD11C2.A2016121.006.2016242201629','MOD11C2.A2016129.006.2016242202036','MOD11C2.A2016137.006.2016242202037','MOD11C2.A2016145.006.2016242202036','MOD11C2.A2016153.006.2016242202517','MOD11C2.A2016161.006.2016242202519','MOD11C2.A2016169.006.2016242202452','MOD11C2.A2016177.006.2016242202906','MOD11C2.A2016185.006.2016242202906','MOD11C2.A2016193.006.2016243201734','MOD11C2.A2016201.006.2016243201734','MOD11C2.A2016209.006.2016243201754','MOD11C2.A2016217.006.2016243201801','MOD11C2.A2016225.006.2016265134738','MOD11C2.A2016233.006.2016265134807','MOD11C2.A2016241.006.2016257120130','MOD11C2.A2016249.006.2016257120208','MOD11C2.A2016257.006.2016271120238','MOD11C2.A2016265.006.2016274183449','MOD11C2.A2016273.006.2016281100939','MOD11C2.A2016281.006.2016292105358','MOD11C2.A2016289.006.2016302012151','MOD11C2.A2016297.006.2016306120153','MOD11C2.A2016305.006.2016313094704','MOD11C2.A2016313.006.2016321103136','MOD11C2.A2016321.006.2016340141813','MOD11C2.A2016329.006.2016340145824','MOD11C2.A2016337.006.2016347144552','MOD11C2.A2016345.006.2016355131206','MOD11C2.A2016353.006.2016362133719','MOD11C2.A2016361.006.2017005151840','MOD11C2.A2017001.006.2017013160908','MOD11C2.A2017009.006.2017020191355','MOD11C2.A2017017.006.2017025093208','MOD11C2.A2017025.006.2017033125003','MOD11C2.A2017033.006.2017041092627','MOD11C2.A2017041.006.2017053170230','MOD11C2.A2017049.006.2017059114655','MOD11C2.A2017057.006.2017065190317','MOD11C2.A2017065.006.2017074143310','MOD11C2.A2017073.006.2017082153636','MOD11C2.A2017081.006.2017090161730','MOD11C2.A2017089.006.2017097200407','MOD11C2.A2017097.006.2017109181521','MOD11C2.A2017105.006.2017118135428','MOD11C2.A2017113.006.2017128142604','MOD11C2.A2017121.006.2017131135238','MOD11C2.A2017129.006.2017139025055','MOD11C2.A2017137.006.2017150121937','MOD11C2.A2017145.006.2017162224657','MOD11C2.A2017153.006.2017163185735','MOD11C2.A2017161.006.2017172161406','MOD11C2.A2017169.006.2017178120344','MOD11C2.A2017177.006.2017187231156','MOD11C2.A2017185.006.2017199162424','MOD11C2.A2017193.006.2017201091253','MOD11C2.A2017201.006.2017212170404','MOD11C2.A2017209.006.2017217101924','MOD11C2.A2017217.006.2017234151702','MOD11C2.A2017225.006.2017233092751','MOD11C2.A2017233.006.2017249205759','MOD11C2.A2017241.006.2017249090516','MOD11C2.A2017249.006.2017262124223','MOD11C2.A2017257.006.2017265090952','MOD11C2.A2017265.006.2017276171238','MOD11C2.A2017273.006.2017281085250','MOD11C2.A2017281.006.2017290124814','MOD11C2.A2017289.006.2017297210133','MOD11C2.A2017297.006.2017310190146','MOD11C2.A2017305.006.2017313085355','MOD11C2.A2017313.006.2017325161755','MOD11C2.A2017321.006.2017331211859','MOD11C2.A2017329.006.2017337153924','MOD11C2.A2017337.006.2017345204221','MOD11C2.A2017345.006.2017354004056','MOD11C2.A2017353.006.2017361091659','MOD11C2.A2017361.006.2018004162642')

**Input references**

This is the MODIS stack catalogue references

In [ ]:
input_references = tuple(['https://catalog.terradue.com/modis/search?uid={0}'.format(pid) for pid in input_identifiers])

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data/modis'

#### Aux folders

In [ ]:
output_folder = ''

In [ ]:
temp_folder = 'temp'

#### Import Modules

In [ ]:
import os
import shutil
import datetime
import sys

#from datetime import datetime, timedelta

try:
    import gdal
except ImportError:
    from osgeo import gdal
import glob

import h5py

import numpy as np

# whittaker engine

sys.path.append(os.getcwd())
sys.path.append('/application/notebook/libexec/')
from modis import ModisRawH5, ModisSmoothH5, ModisMosaic

#### Auxiliary methods

In [ ]:
# remove contents of a given folder
# used to clean a temporary folder
def rm_cfolder(folder):
    #folder = '/path/to/folder'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)



def get_formatted_date(datetime_str):
    date = datetime.datetime.strftime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    
    first_date = datetime.datetime(year=first_date.year, month=first_date.month, day=first_date.day)
    first_date = first_date + datetime.timedelta(days=0, hours=0, minutes=0, seconds=0)
    first_date = get_formatted_date(first_date)
    
    last_date = datetime.datetime(year=last_date.year, month=last_date.month, day=last_date.day)
    last_date = last_date + datetime.timedelta(days=0, hours=23, minutes=59, seconds=59)
    last_date = get_formatted_date(last_date)
    

    
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

#### Auxiliary folders

In [ ]:

if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

#### Collect raw MODIS data into HDF5 file

In [ ]:
modis_abs_paths = [os.path.join(data_path, in_id + '.hdf' ) for in_id in input_identifiers]


try:
    rh5 = ModisRawH5(modis_abs_paths,
                     vam_product_code='LTD',
                     targetdir=temp_folder,
                     interleave=False)
    if not rh5.exists:
        rh5.create(compression='gzip',
                   chunk=None)
    rh5.update()
    
except Exception as e: # pylint: disable=unused-variable
    #print('\nError processing product {}, product code {}. \n\n Traceback:\n'.format(rh5.product, vam_product_code))
    print('something went wrong!!')
    pass


#modis_abs_paths

In [ ]:
#modis_abs_paths

#### Smooth raw MODIS HDF5 file

In [ ]:

h5file = os.path.join(temp_folder, 'LTD', os.listdir(temp_folder + '/LTD')[0] )

smt_h5 = ModisSmoothH5(rawfile=h5file,
                       #startdate='2017001', # Startdate for temporal interpolation (format YYYY-MM-DD or YYYYJJJ)
                       tempint=10,
                       targetdir=temp_folder,
                       nworkers=2)

if not smt_h5.exists:
    smt_h5.create()

#smt_h5.ws2d_sgrid()


# p value
p = 0.90

#srange = [-1, 1, 0.2]
#srange = np.linspace(float(srange[0]),
#                     float(srange[1]),
#                     abs((float(srange[0])-float(srange[1])))/float(srange[2]) + 1.0)

smt_h5.ws2d_vc(None, p)

#### Convert HDF5 to TIFF

In [ ]:

h5_filepath = glob.glob(temp_folder + '/' + '*.TDT.h5')[0]
    

dates_str = []
with h5py.File(h5_filepath, 'r') as h5f:
    dset = h5f.get('data')
    dates = h5f.get('dates')
    
    dates_str = [d for d in dates]
    
    dim = dset.shape
    startdate = dates[0].decode()
    enddate = dates[-1].decode()
    temporalresolution = dset.attrs['temporalresolution']
    resolution = dset.attrs['resolution']
    nodata_value = dset.attrs['nodata']
    processing_timestamp = dset.attrs['processingtimestamp']
    ncols = dset.attrs['RasterXSize'].item()
    nrows = dset.attrs['RasterYSize'].item()



h5files_select_vpc = [h5_filepath]

mosaic = ModisMosaic(files=h5files_select_vpc,
                     datemin=datetime.date(2000, 1, 1).strftime('%Y%m'),
                     datemax=datetime.date.today().strftime('%Y%m'),
                     global_flag=True)
dset = 'data'


filenames = []
file_dates = []
for ix in mosaic.temp_index:
       
    datetime_object =  datetime.datetime.strptime(dates_str[ix], '%Y%j')
        
    filename = os.path.join(output_folder, 'modis_lst_10d_' + str(datetime_object.date()) + '.tif' )
    
    
    filenames.append(filename)
    file_dates.append(datetime_object.date())
    
    print('Processing file {}'.format(filename))
    
    with mosaic.get_raster(dset, ix) as mosaic_ropen:
        #try:
        wopt = gdal.WarpOptions(dstSRS='EPSG:4326',
                                    outputType=mosaic_ropen.dt_gdal[0],
                                    xRes=mosaic_ropen.resolution_degrees,
                                    yRes=mosaic_ropen.resolution_degrees,
                                    srcNodata=mosaic.nodata,
                                    dstNodata=mosaic.nodata,
                                    resampleAlg='near',
                                    multithread=True,
                                    creationOptions=['COMPRESS=LZW', 'PREDICTOR=2'],)

        _ = gdal.Warp(filename,
                          mosaic_ropen.raster,
                          options=wopt)

        _ = None
        #except Exception as e:
        #    print('erro')
            #print('Error while reading {} data for {}! Please check if dataset exits within file. \n\n Error message:\n\n {}'.format(args.dataset, filename, e))
    
    del mosaic_ropen
    

    

filenames


#### write properties files

In [ ]:

for output_name,file_date in zip(filenames,file_dates):
    
    print(file_date)
    
    # all world
    region_of_interest = 'POLYGON((-179.9999 89.9999, 179.9999 89.9999, 179.9999 -89.9999, -179.9999 -89.9999, -179.9999 89.9999))'
    
    write_properties_file(output_name, file_date, file_date, region_of_interest)

#### Remove temporay files and folders

In [ ]:
rm_cfolder(temp_folder)

os.rmdir(temp_folder)